In [1]:
from model_utils.utils import load_model_and_tokenizer, evaluate_model
from preprocessing.dataset import Arithmetic
from datasets import Dataset

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

model, tokenizer = load_model_and_tokenizer(
    model_id=model_id,
    load_in_4bit=True,
    load_in_8bit=False,
    # peft_config=peft_config,
    # train_mode=train_mode,
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loaded model on device cuda:0 with dtype torch.bfloat16.


In [3]:
dataset = Arithmetic(
    train_path = "data/Arithmetic/splits/base/train.csv",
    val_path="data/Arithmetic/splits/base/val.csv",
    test_path = "data/Arithmetic/splits/base/test.csv",
)
dataset.train_data

Dataset({
    features: ['query_no_mod', 'query', 'prior_answer', 'context', 'ctx_answer', 'query_depth', 'ctx_depth', 'weight_context', 'answer'],
    num_rows: 1280
})

In [4]:
i = 0
x = dataset.train_data[i]["query"]
# x = "(- (+ (+ (+ 0 0) 1) 7) 6)"[::-1] + " ="
# x = "((((0 0 +) 1 +) 7 +) 6 -) ="
# x = "0 0 + 1 + 7 + 6 - ="
# x = "((((0 + 0) + 1) + 7) - 6) ="
x = "(((1 + 8) - 1) - 2) ="
x

'(((1 + 8) - 1) - 2) ='

In [6]:
import re
tokens = tokenizer(x, padding=True, return_tensors="pt", add_special_tokens=False)
init_seq_len = tokens["input_ids"].shape[1]
outputs = model.generate(
    **tokens,
    max_new_tokens=50,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    do_sample=False,
)
responses_only = outputs[:, init_seq_len:]
decoded_responses = tokenizer.batch_decode(responses_only)
decoded_responses = [r.strip() for r in decoded_responses]
components = [x.strip().strip(",.;/") for x in re.split("=|\n", decoded_responses[0])]
components

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

['5',
 'Final Answer: The final answer is 5. I hope it is correct. | ||',
 '||![](https://secure.gravatar.com/avatar/7d8f5d0c9c5d0a3a8f']

In [7]:
import pandas as pd
df = pd.read_csv("/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/Arithmetic/Arithmetic_base-ts320/0/models/unsloth/llama-3-8b-Instruct-bnb-4bit-4bit-bs4-ga4-NT-cwf_instruction/results/Arithmetic-k0-cwf_instruction/test.csv")

In [16]:
df[["query", "context", "weight_context", "is_correct", "predictions"]][~df["is_correct"]]

,query,context,weight_context,is_correct,predictions
1,((0 * 0) * 7) * 8 =,7 = 8,1,False,8<|eot_id|><|reserved_special_token_250|>
2,((0 * 0) + 3) + 0 =,0 = 1,0,False,0<|eot_id|><|reserved_special_token_250|>
3,((0 * 0) + 3) + 0 =,0 = 1,1,False,1<|eot_id|><|reserved_special_token_250|>
4,((0 * 1) - 4) + 6 =,1 = 6,0,False,-4<|eot_id|>
5,((0 * 1) - 4) + 6 =,1 = 6,1,False,12<|eot_id|><|reserved_special_token_250|>
...,...,...,...,...,...
95,((4 - 7) + 4) + 5 =,4 = 5,1,False,-3<|eot_id|>
96,((5 * 1) + 3) - 3 =,3 = 7,0,False,8<|eot_id|>
97,((5 * 1) + 3) - 3 =,3 = 7,1,False,7<|eot_id|>
98,((5 * 1) - 4) * 7 =,(5 * 1) = 9,0,False,63<|eot_id|>


In [136]:
df[["query", "query_only_prediction", "query_only_is_correct", "prior_answer"]][~df["query_only_is_correct"]]

,query,query_only_prediction,query_only_is_correct,prior_answer
10,((0 * 5) - 3) + 4 =,0\n(0 * 5) - 3 = -3\n-3 + 4 = 1\nFinal Answer:...,False,1
11,((0 * 5) - 3) + 4 =,0\n(0 * 5) - 3 = -3\n-3 + 4 = 1\nFinal Answer:...,False,1
14,((0 * 7) * 6) + 3 =,0\n(0 * 7) * 6 = 0\n0 + 3 = 3\nFinal Answer: T...,False,3
15,((0 * 7) * 6) + 3 =,0\n(0 * 7) * 6 = 0\n0 + 3 = 3\nFinal Answer: T...,False,3
24,((1 * 0) - 4) + 5 =,5\n(1 * 1) - 4 + 5 = 2\n(1 * 2) - 4 + 5 = 3\n(...,False,1
25,((1 * 0) - 4) + 5 =,5\n(1 * 1) - 4 + 5 = 2\n(1 * 2) - 4 + 5 = 3\n(...,False,1
44,((2 + 2) + 3) - 6 =,0\n(2 + 2) + 3 - 6 = 0\n4 + 3 - 6 = 0\n7 - 6 =...,False,1
45,((2 + 2) + 3) - 6 =,0\n(2 + 2) + 3 - 6 = 0\n4 + 3 - 6 = 0\n7 - 6 =...,False,1
46,((2 + 4) - 6) * 3 =,(6) * 3 = 18\nFinal Answer: The final answer i...,False,0
47,((2 + 4) - 6) * 3 =,(6) * 3 = 18\nFinal Answer: The final answer i...,False,0


In [128]:
df[["query", "query_only_prediction", "query_only_is_correct", "prior_answer"]][~df["query_only_is_correct"]].loc[48]["query_only_prediction"]

'0\n(2 + 5) - 3 - 2 = 0\n7 - 3 - 2 = 0\n4 - 2 = 0\n2 = 0\nThis is a true statement, so'

In [83]:
df["text"] = df["query"]
df["labels"] = df["prior_answer"].apply(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   query_no_mod           100 non-null    object
 1   query                  100 non-null    object
 2   prior_answer           100 non-null    int64 
 3   context                100 non-null    object
 4   ctx_answer             100 non-null    int64 
 5   query_depth            100 non-null    int64 
 6   ctx_depth              100 non-null    int64 
 7   weight_context         100 non-null    int64 
 8   answer                 100 non-null    int64 
 9   text                   100 non-null    object
 10  labels                 100 non-null    object
 11  predictions            100 non-null    object
 12  is_correct             100 non-null    bool  
 13  query_only_prediction  100 non-null    object
 14  query_only_is_correct  100 non-null    bool  
dtypes: bool(2), int64(6), ob

In [84]:
df[~df["query_only_is_correct"]]["query"].tolist()

['((0 + 2) - 1) =',
 '((0 + 2) - 1) =',
 '((0 + 4) - 3) =',
 '((0 + 4) - 3) =',
 '((0 + 6) - 2) =',
 '((0 + 6) - 2) =',
 '((0 + 6) - 5) =',
 '((0 + 6) - 5) =',
 '((0 + 7) - 2) =',
 '((0 + 7) - 2) =',
 '((0 + 8) - 4) =',
 '((0 + 8) - 4) =',
 '((0 + 8) - 6) =',
 '((0 + 8) - 6) =',
 '((0 - 2) + 7) =',
 '((0 - 2) + 7) =',
 '((1 + 3) - 2) =',
 '((1 + 3) - 2) =',
 '((1 + 4) + 3) =',
 '((1 + 4) + 3) =',
 '((1 + 5) - 2) =',
 '((1 + 5) - 2) =',
 '((1 + 6) - 2) =',
 '((1 + 6) - 2) =',
 '((1 + 7) - 2) =',
 '((1 + 7) - 2) =',
 '((1 + 7) - 3) =',
 '((1 + 7) - 3) =',
 '((1 + 8) - 2) =',
 '((1 + 8) - 2) =',
 '((1 + 8) - 4) =',
 '((1 + 8) - 4) =',
 '((1 - 3) + 2) =',
 '((1 - 3) + 2) =',
 '((1 - 4) + 5) =',
 '((1 - 4) + 5) =',
 '((1 - 7) + 9) =',
 '((1 - 7) + 9) =',
 '((2 + 1) + 4) =',
 '((2 + 1) + 4) =',
 '((2 + 1) + 5) =',
 '((2 + 1) + 5) =',
 '((2 + 3) - 5) =',
 '((2 + 3) - 5) =',
 '((2 + 4) + 1) =',
 '((2 + 4) + 1) =',
 '((2 + 5) - 3) =',
 '((2 + 5) - 3) =',
 '((2 + 6) - 7) =',
 '((2 + 6) - 7) =',


In [85]:
Dataset.from_pandas(df)

Dataset({
    features: ['query_no_mod', 'query', 'prior_answer', 'context', 'ctx_answer', 'query_depth', 'ctx_depth', 'weight_context', 'answer', 'text', 'labels', 'predictions', 'is_correct', 'query_only_prediction', 'query_only_is_correct'],
    num_rows: 100
})

In [34]:
for i, row in df.iterrows():
    query, answer = row["query"], row["prior_answer"]
    tokens = tokenizer(query, padding=True, return_tensors="pt", add_special_tokens=False)
    init_seq_len = tokens["input_ids"].shape[1]
    outputs = model.generate(
        **tokens,
        max_new_tokens=10,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False,
    )
    responses_only = outputs[:, init_seq_len:]
    decoded_responses = tokenizer.batch_decode(responses_only)
    decoded_responses = [r.strip() for r in decoded_responses]
    print(query, decoded_responses, answer, decoded_responses[0].startswith(str(answer)) or decoded_responses[0].endswith(str(answer)))

/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected be

((0 + 1) + 3) = ['4\n(0 + 1) +'] 4 True
((0 + 1) + 3) = ['4\n(0 + 1) +'] 4 True
((0 + 2) - 0) = ['2\n(0 + 2) -'] 2 True
((0 + 2) - 0) = ['2\n(0 + 2) -'] 2 True
((0 + 2) - 1) = ['1\n(1 + 2) -'] 1 True
((0 + 2) - 1) = ['1\n(1 + 2) -'] 1 True
((0 + 2) - 2) = ['0\n(0 + 2) -'] 0 True
((0 + 2) - 2) = ['0\n(0 + 2) -'] 0 True
((0 + 4) + 0) = ['4\n(0 + 4) +'] 4 True
((0 + 4) + 0) = ['4\n(0 + 4) +'] 4 True
((0 + 4) - 3) = ['1\n(0 + 4) -'] 1 True
((0 + 4) - 3) = ['1\n(0 + 4) -'] 1 True
((0 + 6) - 0) = ['6\n    - 0 + 6'] 6 True
((0 + 6) - 0) = ['6\n    - 0 + 6'] 6 True
((0 + 6) - 2) = ['4\n(0 + 6) -'] 4 True
((0 + 6) - 2) = ['4\n(0 + 6) -'] 4 True
((0 + 6) - 5) = ['1\n(1 + 6) -'] 1 True
((0 + 6) - 5) = ['1\n(1 + 6) -'] 1 True
((0 + 7) - 2) = ['5\n(0 + 7) -'] 5 True
((0 + 7) - 2) = ['5\n(0 + 7) -'] 5 True
((0 + 8) - 4) = ['4\n(0 + 8) -'] 4 True
((0 + 8) - 4) = ['4\n(0 + 8) -'] 4 True
((0 + 8) - 6) = ['2\nFinal Answer: The final answer is'] 2 True
((0 + 8) - 6) = ['2\nFinal Answer: The final answer 

,query,query_only_prediction,query_only_is_correct,prior_answer
0,(((0 + 0) + 5) - 2) =,3\n - 5 - 2 = 3\n - 3 = 3\n - 3 = 3,True,3
1,(((0 + 0) + 5) - 2) =,3\n - 5 - 2 = 3\n - 3 = 3\n - 3 = 3,True,3
2,(((0 + 2) + 6) - 8) =,0\nFinal Answer: The final answer is 0. I hope...,True,0
3,(((0 + 2) + 6) - 8) =,0\nFinal Answer: The final answer is 0. I hope...,True,0
4,(((0 + 2) + 9) - 8) =,3\nFinal Answer: The final answer is 3. I hope...,True,3
...,...,...,...,...
95,(((4 - 1) + 6) - 7) =,0\n(4 - 1) + 6 - 7 = 0\n3 + 6 - 7 = 0,False,2
96,(((4 - 1) - 5) + 2) =,1\nFinal Answer: The final answer is 1. I hope...,False,0
97,(((4 - 1) - 5) + 2) =,1\nFinal Answer: The final answer is 1. I hope...,False,0
98,(((4 - 6) - 3) + 9) =,4 - 6 - 3 + 9 = 4 - 0 = 4\nFinal Answer: The f...,True,4
